

# Fitting X-ray Spectrum + Opt/UV SED with diskSED

===========================================================

This notebook will go over how to load, configure, and fit (in a Bayesian framework) the full SED. It will make use of three pieces of software:
* pyXSPEC: https://heasarc.gsfc.nasa.gov/xanadu/xspec/python/html/ 
* Bayesian X-ray Analysis (BXA): https://johannesbuchner.github.io/BXA/index.html
* UltraNest: https://johannesbuchner.github.io/UltraNest/readme.html
  
Some reading on all is desired, we are going to cover just the basics below.

Relevant bibliography for introduction to fitting X-ray data in a Bayesian framework: https://arxiv.org/abs/2309.05705

Relevant bibliography on why reduced chi^2 is not a great method for this: https://arxiv.org/abs/1012.3754

> **Note:**  In principle, one could use another Bayesian parameter inference software/technique. Here, we are using UltraNest, which employs the nested sampling method, but one could also use, e.g., emcee ( https://emcee.readthedocs.io/en/stable/ ), which employs Markov Chain Monte Carlo (MCMC) methods.







In [2]:
# General imports
import os
import numpy as np
from astropy.cosmology import FlatLambdaCDM

# Setting cosmology
cosmo = FlatLambdaCDM(H0=73, Om0=0.3, Tcmb0=2.725)

# Xspec, BXA, and UltraNest imports and config
from xspec import *
Xset.chatter = 10  # Controls pyXPSEC output
Xset.cosmo = "73"  # Cosmology for pyXPSEC
import bxa.xspec as bxa
bxa.BXASolver.allowed_stats.append("chi")  # Enable Gaussian statistics
from bxa.xspec.solver import set_parameters  # For later use
Fit.statMethod = "chi"  # Set pyXPSEC statistics to Gaussian
Fit.query = "yes"
Plot.device = "/xs"  # Plotting console for pyXPSEC
Plot.yLog = True
Plot.background = True

# Load diskSED model
from diskSED import diskSED_info, diskSED
AllModels.addPyMod(diskSED, diskSED_info(), 'add')  # Additive model

# Load reddenSF model (Calzetti et al. attenuation law)
from diskSED import reddenSF_info, reddenSF
AllModels.addPyMod(reddenSF, reddenSF_info(), 'mul')  # Multiplicative model

# Load reddenCCM model (CCM extinction law) This is the same as the XSPEC native 'redden' but extended to the Lyman limit.
from diskSED import reddenCCM_info, reddenCCM
AllModels.addPyMod(reddenCCM, reddenCCM_info(), 'mul')  # Multiplicative model

Default fit statistic is set to: Chi-Squared
   This will apply to all current and newly loaded spectra.


In [3]:
# Current working directory
cwd = os.getcwd()

# Source redshift
z = 0.0206

# Galactic color excess at 14li's position 
E_BV_G = 0.022

# Galactic column density at 14li's position (units of 10^22)
N_H_G = 1.95E+20 / 1e22

In [4]:
# Quick cleaning (in case something has been loaded already)
AllData.clear()
AllModels.clear()
AllData.clear()

Plot.xAxis = "keV"
# Set energy bins for the model (working from 5e-4 keV to 2 keV with 400 log bins)
AllModels.setEnergies('0.0005 2.0 400 log')

# Load X-ray and UV/optical data
E_lim = [0.25, 0.9]  # Energy limits for X-ray spectrum, use wathever criteria you think is resonable
os.chdir(os.path.join(cwd, 'data', 'SED'))
x_ray_file = '14li_E1_grp.fits'
uvopt_file = 'uvopt.pha'
AllData('1:1 ' + str(x_ray_file) + ' 1:2 ' + str(uvopt_file))

# Ignore bad channels
Xray = AllData(1)
Xray.response.arf = '14li_E1_arf.fits' # Sometimes we need to add the arf manually, it fails to find automatically.
Xray.ignore('0.0-' + str(E_lim[0]) + ' ' + str(E_lim[1]) + '-**')
uvopt = AllData(2)
uvopt.response = 'uvopt.rmf'
AllData.ignore("bad")

#If you had another epoch you could use this to load it:
'''
E_lim  = [0.25, .8]
os.chdir(os.path.join(cwd, 'data', 'SED2'))
x_ray_file2 = '14li_E2_grp.fits'
uvopt_file2 = 'uvopt2.pha'
AllData('2:3 '+str(x_ray_file2) + ' 2:4 ' +str(uvopt_file2))
Xray2 =  AllData(3)
Xray2.ignore('0.0-'+ str(E_lim[0])+ ' '+ str(E_lim[1])+ '-**')
uvopt2 = AllData(4)
uvopt2.response = 'uvopt2.rmf'
'''

# Let's Plot the data
Plot("data") # Yes, the units look weird this way. But is just to make sure, it is all loaded. You see an X-ray spectrum (with background) and an opt/UV SED.

os.chdir(cwd)


Models will now use energy array created from:
   0.0005 - 2   400 log bins

***Warning: Unrecognized grouping for channel(s). It/they will be reset to 1.

2 spectra  in use
 
Spectral Data File: 14li_E1_grp.fits  Spectrum 1
Net count rate (cts/s) for Spectrum:1  1.528e+00 +/- 6.581e-03 (98.9 % total)
 Assigned to Data Group 1 and Plot Group 1
  Noticed Channels:  1-1189
  Telescope: XMM Instrument: EPN  Channel Type: PI
  Exposure Time: 3.574e+04 sec
 Using fit statistic: chi
 Using Background File                14li_E1_bkg.fits
  Background Exposure Time: 3.574e+04 sec
 Using Response (RMF) File            14li_E1_rmf.fits for Source 1

Spectral Data File: uvopt.pha  Spectrum 2
Net count rate (cts/s) for Spectrum:2  6.010e-02 +/- 5.473e-03
 Assigned to Data Group 1 and Plot Group 2
  Noticed Channels:  1-4
  Telescope:  Instrument:   Channel Type: PI
  Exposure Time: 1 sec
 Using fit statistic: chi
 Using Response (RMF) File            uvopt.rmf for Source 1

Arf successfully loade

In [5]:
# Set the model

m = Model('phabs*redden*zashift*phabs*reddenSF*(diskSED)') 
# Please read arXiv:2408.17296 for a detailed description:
# phabs*reddenCCM accounts for Galactic X-ray neutral gas absorption and Galactic dust extinction, respectively.
# zashift redshifts the model.
# phabs*reddenSF accounts for intrinsic X-ray neutral gas absorption and intrinsic dust attenuation, respectively.
# Note: We use phabs instead of, e.g., tbabs, because tbabs corrects down to UV wavelengths (not only X-ray) and using it would result in double correction in the UV.

AllModels.show() # The model components are shown below.


Model phabs<1>*redden<2>*zashift<3>*phabs<4>*reddenSF<5>*diskSED<6> Source No.: 1   Active/On
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   phabs      nH         10^22    1.00000      +/-  0.0          
   2    2   redden     E_BmV               5.00000E-02  +/-  0.0          
   3    3   zashift    Redshift            0.0          frozen
   4    4   phabs      nH         10^22    1.00000      +/-  0.0          
   5    5   reddenSF   ebmv                1.00000E-03  +/-  0.0          
   6    6   diskSED    R_in*      km       1.00000E+06  +/-  0.0          
   7    6   diskSED    T_p        Kelvin   5.00000E+05  +/-  0.0          
   8    6   diskSED    R_ratio             100.000      +/-  0.0          
   9    6   diskSED    D_Mpc      Mpc      10.0000      +/-  0.0          
  10    6   diskSED    norm                1.00000      +/-  0.0          
________________________________________________________________________


Fit statistic  : Chi-Squared    

In [6]:
# Let's edit these parameters, fix what needs to be fixed, and add reasonable ranges to those that will be fitted.

# phabs Nh
AllModels(1)(1).values = (N_H_G, -1)  # Fixing

# reddenCCM
AllModels(1)(2).values = (E_BV_G, -1) # This assumes a Galactic Gas-to-dust Ratio with R_v  = 3.1. See comments below. 

# zashift Redshift
AllModels(1)(3).values = (z, -1)  # Fixing

# phabs Nh
AllModels(1)(4).values = (0.04, 0.001, 0.02, 0.02, 0.1, 0.1)  
# Check the pyxspec manual for the meaning of each of the six entries. The first entry is the initial guess, but that is not used in a Bayesian fitting.

# redden
AllModels(1)(5).link = '4 * 1e22 / 8.9505e21'  
# Linking intrinsic E(B-V) (parameter 5) to intrinsic N_H (parameter 4). 
# Assumes a Galactic Gas-to-Dust Ratio (https://arxiv.org/abs/0903.2057) with R_v = 4.05 (consistent with Calzetti's law).
# You could also make this free or use, e.g., a Gaussian prior centered around an independent estimate 
# for the intrinsic E(B-V), such as from the Balmer decrement of the nuclear optical spectrum of the source.

# Parameters for diskSED. Feel free to modify give the expectations from your source/observations; note that larger ranges will increase convergence time.

# diskSED R_in*: Expected range for the Rin* parameter in km.
AllModels(1)(6).values = (3e7, 10, 3e7, 3e7, 1e8, 1e8)

# diskSED T_p: Expected range for inner disk temperature in Kelvin.
AllModels(1)(7).values = (2e5, 1e-2, 1e5, 1e5, 1e6, 1e6)

# diskSED R_ratio: Expected outer-to-inner radius ratio range.
AllModels(1)(8).values = (10, 1e-1, 5, 5, 30, 30)

# diskSED Distance: Distance to the source in Mpc. Needs to be fixed.
AllModels(1)(9).values = (cosmo.luminosity_distance(z).to('Mpc').value, -1)

# diskSED norm: This must always be set to 1.
AllModels(1)(10).values = (1, -1)  
# Rin* is already the normalization parameter of the model.

# Check the model below; there should be exactly 4 free parameters, and 2 Tied.
AllModels.show()



Fit statistic  : Chi-Squared                23297.00     using 8 bins, spectrum 1.
                 Chi-Squared                  123.59     using 4 bins, spectrum 2.
Total fit statistic                         23420.59     with 4 d.o.f.

Test statistic : Chi-Squared                23420.59     using 12 bins.
 Null hypothesis probability of 0.00e+00 with 4 degrees of freedom
 Current data and model not fit yet.

Fit statistic  : Chi-Squared                23297.00     using 8 bins, spectrum 1.
                 Chi-Squared                  287.08     using 4 bins, spectrum 2.
Total fit statistic                         23584.09     with 5 d.o.f.

Test statistic : Chi-Squared                23584.09     using 12 bins.
 Null hypothesis probability of 0.00e+00 with 5 degrees of freedom
 Current data and model not fit yet.

Fit statistic  : Chi-Squared                23477.64     using 8 bins, spectrum 1.
                 Chi-Squared                  293.62     using 4 bins, spectrum 2.
Tot

# Preparing the statistics for the fitting:

Here we have two options:

i) use BXA to create the priors, prior transformations (it has some option: e.g. uniform, log-uniform, and gaussian) and the likehood function. This should help undertand BXA better: https://peterboorman.com/xsf22

ii) Or create our own prior, prior transformation and likehood functions from scratch, details on how to to that are avalible in the UltraNest page: https://johannesbuchner.github.io/UltraNest/using-ultranest.html

I will examplify both below, for very simple linear/log-linear priors and Gaussian likehood. In general we will use the BXA tools, we may want to do from scratch if we want special priors (besides uniform, log-uniform, or Gaussian) which are not included in BXA, or if we wanna make more complex likehood functions, e.g. add upper limits, or combine Gaussian and Poisson statistics.

In [7]:
# Using BXA:

# These creates the priors
prior_NH = bxa.create_uniform_prior_for(AllModels(1), AllModels(1)(4)) # The number 1 is the data group (here we only have 1), while the other number is the paramter order, e.g. 4, see above. See BXA tutorial for details.
prior_R = bxa.create_loguniform_prior_for(AllModels(1), AllModels(1)(6))
prior_T = bxa.create_uniform_prior_for(AllModels(1), AllModels(1)(7))
prior_R_ratio = bxa.create_uniform_prior_for(AllModels(1), AllModels(1)(8))

# This creates all the rest
solver = bxa.BXASolver(transformations=[prior_NH, prior_R, prior_T, prior_R_ratio])

# Define/load here
paramnames_bxa = solver.paramnames
prior_bxa  = solver.prior_function
loglike_bxa  = solver.log_likelihood
prior_transform_bxa = solver.transformations

print('------------------------------------------')
print(paramnames_bxa) # This is just a list with name of paramters

  uniform prior for nH between 0.020000 and 0.100000 
  jeffreys prior for R_in* between 3.000000e+07 and 1.000000e+08 
  uniform prior for T_p between 100000.000000 and 1000000.000000 
  uniform prior for R_ratio between 5.000000 and 30.000000 
------------------------------------------
['nH', 'log(R_in*)', 'T_p', 'R_ratio']


In [8]:
# Creating from scratch. THIS IS ADVANCED USE, skip this cell and use the BXA ready functions, if you just wanna do your first run. 

# Paremeters names
paramnames = ['nH', 'log(R_in*)', 'T_p', 'R_ratio']

# Creating priors

def create_uniform_prior_for(model, par):
	pval, pdelta, pmin, pbottom, ptop, pmax = par.values
	low = float(pmin)
	spread = float(pmax - pmin)
	def uniform_transform(x): return x * spread + low
	return dict(model=model, index=par._Parameter__index, name=par.name, transform=uniform_transform, aftertransform=lambda x: x)
    

def create_loguniform_prior_for(model, par):
	pval, pdelta, pmin, pbottom, ptop, pmax = par.values
	low = np.log10(pmin)
	spread = np.log10(pmax) - np.log10(pmin)
	def log_transform(x): return x * spread + low
	def log_after_transform(x): return 10**x
	return dict(model=model, index=par._Parameter__index, name='log(%s)' % par.name, transform=log_transform, aftertransform=log_after_transform)


prior_NH = create_uniform_prior_for(AllModels(1), AllModels(1)(4))
prior_R = create_loguniform_prior_for(AllModels(1), AllModels(1)(6))
prior_T = create_uniform_prior_for(AllModels(1), AllModels(1)(7))
prior_R_ratio = create_uniform_prior_for(AllModels(1), AllModels(1)(8))
prior_transform=[prior_NH, prior_R,prior_T,prior_R_ratio ]

# Creating prior transformations (see UltraNest tutorial for deeper undertanding)

def create_prior_function(transformations):
	"""
	Create a single prior transformation function from a list of
	transformations for each parameter. This assumes the priors factorize.
	"""

	def prior(cube):
		params = cube.copy()
		for i, t in enumerate(transformations):
			transform = t['transform']
			params[i] = transform(cube[i])
		return params

	return prior  

prior = create_prior_function(prior_transform)

# Creating a log-likehood function
def log_likelihood(params):
    """ returns -0.5 of the fit statistic."""
    set_parameters(transformations=prior_transform, values=params) # This simple set all the free prameter to a given array 'params'
    like = -0.5 * Fit.statistic # Fit statistics here, simple return the current statistics of the model , i.e. SUM ((data-model)/error)^2
    if not np.isfinite(like): # Just check if it is finite
        return -1e100
    return like

loglike = log_likelihood

In [10]:
# OK, now we can run some Baeysian inference. I will use UltraNest.
import ultranest
from ultranest import ReactiveNestedSampler
import pandas as pd
# We wanna make pyXSPEC quiet now. So just Ultranest can display progress.
Xset.chatter = 1

os.chdir(os.path.join(cwd, 'data')) # make sure we are where we want.
outputfiles_basename = 'diskSED_run' # Name of the folder we gonna save the sampling results

#picking BXA funcitons
sampler = ReactiveNestedSampler(paramnames_bxa, loglike_bxa, prior_bxa, log_dir=outputfiles_basename, resume='overwrite')

#picking our own funcitons
#sampler = ReactiveNestedSampler(paramnames, loglike, prior, log_dir=outputfiles_basename, resume='overwrite')

result = sampler.run(frac_remain=1e-2, min_num_live_points=200,min_ess=400) # See UltraNest for details on what the varaibles mean. These are standard values. 
# Some intuition: 
# decreasing frac_remain (e.g. to 5e-2, or 1e-1) wil make the sampling finish earlier, running longer does not mean necessarily a better fit.
# decreasing min_num_live_points decreases the 'resolution' of the sampling, but makes the computations faster.

sampler.print_results() # print final results
sampler.plot() # makes some standard corner, and tracer plots

posterior_df = pd.DataFrame(data=result["samples"], columns=solver.paramnames)
posterior_df.to_csv(os.path.join(cwd, 'data', "%s/posterior.csv" %(outputfiles_basename)), index=False) # Just saving the posterior into a .csv file, cause it is easier.

# This will run and save the results inside a 'diskSED_run' folder.
# It will take time (10-20 min). If you wanna speed up things, e.g. by running in multiple cores. There will be a 'fitting.py' script, which we can parallelize with MPI (https://www.open-mpi.org).
# In the terminal, inside the examples folder, just do 'mpiexec -N X python3 fitting.py' where X is the number of cores you wanna run the sampling.

[ultranest] Sampling 200 live points from prior ...


[ultranest] Explored until L=-4  33 [-4.4753..-4.4752]*| it/evals=3996/12690 eff=31.9936% N=200              00 0 0   
[ultranest] Likelihood function evaluations: 12690
[ultranest] Writing samples and results to disk ...
[ultranest] Writing samples and results to disk ... done
[ultranest]   logZ = -19.76 +- 0.1689
[ultranest] Effective samples strategy satisfied (ESS = 1072.7, need >400)
[ultranest] Posterior uncertainty strategy is satisfied (KL: 0.45+-0.07 nat, need <0.50 nat)
[ultranest] Evidency uncertainty strategy is satisfied (dlogz=0.17, need <0.5)
[ultranest]   logZ error budget: single: 0.26 bs:0.17 tail:0.01 total:0.17 required:<0.50
[ultranest] done iterating.

logZ = -19.801 +- 0.376
  single instance: logZ = -19.801 +- 0.260
  bootstrapped   : logZ = -19.763 +- 0.376
  tail           : logZ = +- 0.010
insert order U test : converged: True correlation: inf iterations

    nH                  : 0.0252│ ▁▁ ▁▁▁▁▁▂▂▂▃▄▄▅▇▇▇▇▆▆▅▄▃▃▃▂▂▁▁▁▁▁▁▁ ▁ │0.0624    0.0434 +- 0.0047
    l